# Run a pipeline for a campaign

#### Instructions:

This "notebook" allows a user to work through all the steps in a campaign. Just go through the steps one-by-one. To execute a step, click on it, and then click on the "Run" button. 

You can stop at any point, and then return to the notebook at any later time. If you do, first run the first initialization section.

All the output and plots will stay in between the sessions. To clean everything up, press "Cell" in the menu bar -> "All Output" -> "Clear".

## Initialization. Run every time.

Run it every time the notebook starts.

In [ ]:
%matplotlib inline
import os
import matplotlib
import matplotlib.pyplot as plt
import argparse
import glob
import shutil

Input data

In [ ]:
CAMPAIGN_ID = 7

## Start campaign

Create a new campaign and pick images.

In [ ]:
!pipeline/select_new_campaign.sh --num_images_in_campaign 50 --out_version 1 --campaign_id {CAMPAIGN_ID}

Detect stamps using the best model

In [ ]:
!pipeline/detect_stamps.sh --in_version 1 --out_version 2 --campaign_id {CAMPAIGN_ID}

Detect pages using the best model

In [ ]:
!pipeline/detect_stamps.sh --in_version 2 --out_version 3 --campaign_id {CAMPAIGN_ID}

Crop stamps to run classification

In [ ]:
!pipeline/crop_stamps_for_classification.sh --in_version 3 --campaign_id {CAMPAIGN_ID}

Run the classification

In [ ]:
!pipeline/classify_stamps.sh 

Import the classification results.

In [ ]:
!pipeline/import_classification_results.sh --in_version 3 --out_version 4 --campaign_id {CAMPAIGN_ID}

Visualize ML predictions.

In [ ]:
!pipeline/visualize_ml_predictions.sh --in_version 4 --campaign_id {CAMPAIGN_ID}

## Labeling in Labelme

Export to Labelme

In [ ]:
!pipeline/export_to_labelme.sh --in_version 4 --campaign_id {CAMPAIGN_ID}

Upload to the LabelMe server

In [ ]:
!pipeline/upload_to_labelme_server.sh --name "initial" --campaign_id {CAMPAIGN_ID}

#### Please go to https://vm019.bridges2.psc.edu/ and label data.

Download from the Labelme server

In [ ]:
!pipeline/download_from_labelme_server.sh --name "initial" --campaign_id {CAMPAIGN_ID}

Import labeled data from Labelme

In [ ]:
!pipeline/import_from_labelme.sh --out_version 5 --campaign_id {CAMPAIGN_ID}

Display statistics. It will help to decide if more cleaning or more custom rules are needed.

In [ ]:
!pipeline/display_statitics.sh --in_version 5 --campaign_id {CAMPAIGN_ID}

## Cleaning in Labelme

#### For every round of cleaning, repeat the following steps. 

Every new time you do the cleaning, increment CLEANING_ITERATION.

In [ ]:
CLEANING_ITERATION = 1

Export to labelme as "collage" tiles for cleaning. May take 10-30 mins.

In [ ]:
!pipeline/export_to_labelme_cleaning.sh --in_version 5 --subversion {CLEANING_ITERATION} --campaign_id {CAMPAIGN_ID}

Upload to the Labelme server

In [ ]:
!pipeline/upload_to_labelme_server.sh --name cleaning{CLEANING_ITERATION} --campaign_id {CAMPAIGN_ID}

#### Please go to https://vm019.bridges2.psc.edu/ and clean data.

Download from the Labelme server. **Make sure to first run the cell `CLEANING_ITERATION =` (above).**

In [ ]:
!pipeline/download_from_labelme_server.sh --name cleaning{CLEANING_ITERATION} --campaign_id {CAMPAIGN_ID}

Import from labelme after cleaning

In [ ]:
!pipeline/import_from_labelme_cleaning.sh --in_version 5 --out_version 6 --subversion {CLEANING_ITERATION} --campaign_id {CAMPAIGN_ID}

Apply special rules. Rename class and delete class is currently supported

In [ ]:
!pipeline/custom_cleaning.sh --rename_class "\"{'dog': 'cat', 'sheep': 'panda'}\""  --in_version 6 --out_version 7 --subversion {CLEANING_ITERATION} --campaign_id {CAMPAIGN_ID}

Display statistics. It will help to decide if more cleaning or more custom rules are needed.

In [ ]:
!pipeline/display_statitics.sh --in_version 7 --subversion {CLEANING_ITERATION} --campaign_id {CAMPAIGN_ID}

Finalize cleaned.

In [ ]:
!pipeline/promote_cleaning_iteration.sh --in_version 7 --subversion {CLEANING_ITERATION}

#### No more cleaning after this step.

## Training ML models

Start STAMP detection training. It may take 4-12 hours. Run the next step in parallel to this one.

In [ ]:
!pipeline/start_stamp_detection_training.sh --in_version 7 --campaign_id {CAMPAIGN_ID}

Start PAGE detection training. It may take 4-12 hours. Run it in parallel with the prevoius step.

In [ ]:
!pipeline/start_page_detection_training.sh --in_version 7 --campaign_id {CAMPAIGN_ID}

#### Please wait until the training is complete

Check if all training has completed.

In [ ]:
!pipeline/check_if_all_jobs_have_completed.sh

Finalize STAMP detection training.

In [ ]:
!pipeline/finalize_stamp_detection_training.sh --in_version 7 --campaign_id {CAMPAIGN_ID}

Finalize PAGE detection training.

In [ ]:
!pipeline/finalize_page_detection_training.sh --in_version 7 --campaign_id {CAMPAIGN_ID}

Evaluate detectors trained on previous campaigns on this campaign

In [ ]:
!pipeline/evaluate_previous_detectors_on_this.sh --in_version 7 --campaign_id {CAMPAIGN_ID}

Crop out stamps to train the classification model

In [ ]:
!pipeline/crop_stamps_to_train_classifier.sh --in_version 7 --campaign_id {CAMPAIGN_ID}

Start the classification training

In [ ]:
!pipeline/start_stamp_classification_training.sh --in_version 7 --campaign_id {CAMPAIGN_ID}

Check if all training has completed.

In [ ]:
!pipeline/check_if_all_jobs_have_completed.sh

## Release the MiikeMineStamps dataset

Make a release of MiikeMineStamps dataset. Its README files need to be changed separately manually.

In [ ]:
!pipeline/release_miike_mine_stamp_dataset.sh --in_version 7 --campaign_id {CAMPAIGN_ID}